
# Bevezetés a Python Polars csomag használatába


## 1. Telepítés

Az alap csomag telepítése:
```bash
 pip install polars
```

### 1.1 Opcionális csomagok (függőségek) telepítése

| Tag        | Leírás                                                                                           |
|------------|--------------------------------------------------------------------------------------------------|
| all        | Az összes opcionális függőség telepítése (az összes alábbi)                                       |
| pandas     | Telepítés Pandas-szal az adatok Pandas Dataframe-ekre/sorozatokra való átalakításához            |
| numpy      | Telepítés numpy-val az adatok numpy tömbökké történő átalakításához                               |
| pyarrow    | Adatformátumok olvasása a PyArrow használatával                                                  |
| fsspec     | Távoli fájlrendszerekből való olvasás támogatása                                                 |
| connectorx | SQL adatbázisokból való olvasás támogatása                                                       |
| xlsx2csv   | Excel fájlokból való olvasás támogatása                                                          |
| deltalake  | Delta Lake táblákból való olvasás támogatása                                                     |
| timezone   | Időzóna támogatás, csak akkor szükséges, ha Python<3.9-et használ, vagy Windows rendszeren van   |

Opcionális függőségek részhalmazainak telepítése:  
```bash
 pip install 'polars[pandas,numpy]'
```

Az összes opcionális függőség telepítése:  
```bash
pip install 'polars[all]'
``` 

### 1.2 Konfiguráció

Konfigurációs lehetőségek a hivatalos dokumentációban: [Config](https://docs.pola.rs/api/python/stable/reference/config.html)


In [ ]:
import numpy as np
import polars as pl

pl.Config.set_tbl_rows(3)  # Az alapértelmezett érték 10
pl.Config.set_tbl_cols(None)  # Az alapértelmezett érték 10
pl.Config.set_verbose(active=True)

In [ ]:
import random
import string

nr_cols = 10
nr_rows = 6
string_length = 5
data = {}
for i in range(nr_cols):
    data[f'column_{i}'] = ["".join(random.choice(string.ascii_letters) for c in range(string_length)) \
                           for _ in range(nr_rows)]

df = pl.DataFrame(data)
print(df)

print("Konfigurációs beállítások felülírása:")
with pl.Config(tbl_cols=-1):
    print(df)

## 2. Támogatott adattípusok

Link a hivatalos dokumentáció kapcsolódó oldalához: [Data types](https://docs.pola.rs/api/python/stable/reference/datatypes.html)


In [108]:
df_szemelyek = pl.DataFrame({
    "nev": ["Kovács Jolán", "Nagy János", "Kiss Mária", "Nagy Zoltán"],
    "nem": ["nő", "férfi", "nő", "férfi"],
    "szuletesi_datum": ["1996-05-12", "1987-02-23", "1993-11-30", "1979-09-15"],
    "magassag": [165.5, 180.0, 173.3, 165.0]
}, infer_schema_length=1)

# infer_schema_length: int vagy None  
# A séma kikövetkeztetéséhez beolvasandó maximális sorok száma. Ha None értékre van állítva, akkor a teljes adat beolvasható (ez lassú lehet).

print(df_szemelyek.schema)
print(df_szemelyek.dtypes)

elvart_sema = pl.Schema({
    "x": pl.Float64,
    "y": pl.Float64,
    "szin": pl.Utf8,
    "datum": pl.Date
})
df_pontok = pl.DataFrame({
    "x": [1.1, 2, 3.5, 4, 5],
    "y": [6.5, -4, 3, 2.1, 1],
    "szin": ["piros", "kék", "piros", "sárga", "fekete"],
    "datum": ["2021-01-12", "2021-02-23", "2021-03-30", "2021-04-15", "2021-05-20"]
}, schema=elvart_sema)

print(df_pontok)



Schema({'nev': String, 'nem': String, 'szuletesi_datum': String, 'magassag': Float64})
[String, String, String, Float64]
shape: (5, 4)
┌─────┬──────┬────────┬────────────┐
│ x   ┆ y    ┆ szin   ┆ datum      │
│ --- ┆ ---  ┆ ---    ┆ ---        │
│ f64 ┆ f64  ┆ str    ┆ date       │
╞═════╪══════╪════════╪════════════╡
│ 1.1 ┆ 6.5  ┆ piros  ┆ 2021-01-12 │
│ 2.0 ┆ -4.0 ┆ kék    ┆ 2021-02-23 │
│ 3.5 ┆ 3.0  ┆ piros  ┆ 2021-03-30 │
│ 4.0 ┆ 2.1  ┆ sárga  ┆ 2021-04-15 │
│ 5.0 ┆ 1.0  ┆ fekete ┆ 2021-05-20 │
└─────┴──────┴────────┴────────────┘


 ### 2.1 További adattípusok

| Kategória       | Típus       | Leírás                                                                                                                     | Megjegyzés                                     |
|:----------------|-------------|-----------------------------------------------------------------------------------------------------------------------------|------------------------------------------------|
| Idő és dátum    | Date        | Naptári dátum típus. Az Arrow date32 adat típust használja, napok az 1970-01-01 UNIX epoch óta int32 formátumban.           | -5877641-06-24 és 5879610-09-09 között         |
|                 | Datetime    | Naptári dátum és idő típus. Pontos időbélyeg int64 formátumban kódolva az UNIX epoch óta. Alapértelmezett egység mikroszekundum. |                                                |
|                 | Duration    | Időtartam/különbség típus.                                                                                                  |                                                |
|                 | Time        | Napi idő típus.                                                                                                            |                                                |
| Beágyazott      | Array(*args, **kwargs) | Fix hosszúságú lista típus.                                                                                |                                                |
|                 | List(*args, **kwargs)  | Változó hosszúságú lista típus.                                                                               |                                                |
|                 | Struct(*args, **kwargs) | Struktúra típus.                                                                                             |                                                |
| Egyéb           | Boolean     | Boolean típus, amely 1 bitnyi helyet foglal.                                                                               | Igaz vagy Hamis                                |
|                 | Binary      | Bináris típus változó hosszúságú bájtokkal.                                                                                |                                                |
|                 | Categorical | Egy kategóriák szerinti kódolás sztringek halmazához. Hatékonyabb memóriakezelést tesz lehetővé, ha egy oszlopban kevés egyedi sztring található. |                                                |
|                 | Null        | Null / None értékeket képviselő típus.                                                                                     |                                                |
|                 | Object      | Bármilyen Python objektum csomagolására szolgáló típus.                                                                    | Használata elkerülendő, amennyiben lehetséges. |
|                 | String      | UTF-8 kódolású sztring típus változó hosszúsággal.                                                                         |                                                |
|                 | Unknown     | Típus, amely azokat az adatértékeket képviseli, amelyeket statikusan nem lehetett meghatározni.                             |                                                |



Pár példa a fenti típusok használatára:

In [ ]:
pontok_df = pl.DataFrame({
    "egesz_szamok": [[1,2, -1], [3,4]],
    "tizedes_szamok": [[4.2,-1, 3.1], [3, 1, 0, 2]]
    }, strict=False)
print(pontok_df)

In [107]:
df_tombok = pl.DataFrame(
    [
        pl.Series("tomb_1", [[-1, 2], [2, 4]]),
        pl.Series("tomb2_2", [[1, 7, 1], [2, -1, 0]]),
    ],
    schema={
        "tomb_1": pl.Array(shape=(2,), inner=pl.Int64),
        "tomb_2": pl.Array(shape=(3,), inner=pl.Int64)
    }
)
print(df_tombok)
# korábban "tomb_1": pl.Array(width=2, ez deprecated lett


shape: (2, 2)
┌───────────────┬───────────────┐
│ tomb_1        ┆ tomb_2        │
│ ---           ┆ ---           │
│ array[i64, 2] ┆ array[i64, 3] │
╞═══════════════╪═══════════════╡
│ [-1, 2]       ┆ [1, 7, 1]     │
│ [2, 4]        ┆ [2, -1, 0]    │
└───────────────┴───────────────┘


### 2.2 Konverziók

    

In [ ]:

df = pl.DataFrame({
    "id": [1, 2, 3, 4, 5],
    "meres_1": [6, -1, 3, 9.0, -10],
    "meres_2": ["1.0", "2.0", "3.0", "4.0", "5.0"]
}, strict=False)
print(df)
print(f"Becsült méret: {df.estimated_size('b')} bájt")
#region-> polars.DataFrame.cast() metódus használata
df.with_columns(
    pl.col("id").cast(pl.UInt32),
    pl.col("meres_1").cast(pl.Int32),
    pl.col("meres_2").cast(pl.Float32).cast(pl.Int32)
)

print(df)
print(f"Becsült méret: {df.estimated_size('b')} bájt")
#endregion


In [ ]:
df = pl.DataFrame({
    "id": [1, 2, 3, 4, 5],
    "meres_1": [6, -1, 3, 9.0, -10],
    "meres_2": [1.0, -2.0, 4.0, 5.0, 6.0]
}, strict=False).cast(pl.Int32)
print(df)

## 3. Ismerkedés a polars.DataFrame osztállyal
A `polars.DataFrame` osztály bemutatása a hivatalos dokumentációban:   
[https://docs.pola.rs/api/python/stable/reference/dataframe/index.html](https://docs.pola.rs/api/python/stable/reference/dataframe/index.html)   és [implementációja a Github repóban](https://github.com/pola-rs/polars/blob/py-1.12.0/py-polars/polars/dataframe/frame.py#L191-L11302)  

**Megjegyzés:** Amennyiben a `df.plot(...)` sorokra `ModuleUpgradeRequiredError: altair>=5.4.0 is required for .plot` hibával találkozunk akkor szükséges az `altair` parancs szükséges az alábbi parancs futtatása:
```bash
pip install --upgrade altair
```

In [114]:
#region-> Példa egyszerű Polars dataframe létrehozására, adatok diagramon való megjelenítése
import polars as pl
from datetime import date

df = pl.DataFrame(
    {
        "date": [date(2020, 1, 2), date(2020, 1, 3), date(2020, 1, 4)] * 2,
        "stock": ["a", "a", "a", "b", "b", "b"],
        "price": [1, 4, 6, 1, 5, 2],

    }
)

print(f"A DataFrame sémája: {df.schema}")
print(f"A DataFrameben tárolt adatok megtekintése: {df}")
print("További információk:", df.describe())

df.plot.line(x="date", y="price", color="stock")
#endregion

A DataFrame sémája: Schema({'date': Date, 'stock': String, 'price': Int64})
A DataFrameben tárolt adatok megtekintése: shape: (6, 3)
┌────────────┬───────┬───────┐
│ date       ┆ stock ┆ price │
│ ---        ┆ ---   ┆ ---   │
│ date       ┆ str   ┆ i64   │
╞════════════╪═══════╪═══════╡
│ 2020-01-02 ┆ a     ┆ 1     │
│ 2020-01-03 ┆ a     ┆ 4     │
│ 2020-01-04 ┆ a     ┆ 6     │
│ 2020-01-02 ┆ b     ┆ 1     │
│ 2020-01-03 ┆ b     ┆ 5     │
│ 2020-01-04 ┆ b     ┆ 2     │
└────────────┴───────┴───────┘
További információk: shape: (9, 4)
┌────────────┬─────────────────────┬───────┬──────────┐
│ statistic  ┆ date                ┆ stock ┆ price    │
│ ---        ┆ ---                 ┆ ---   ┆ ---      │
│ str        ┆ str                 ┆ str   ┆ f64      │
╞════════════╪═════════════════════╪═══════╪══════════╡
│ count      ┆ 6                   ┆ 6     ┆ 6.0      │
│ null_count ┆ 0                   ┆ 0     ┆ 0.0      │
│ mean       ┆ 2020-01-03 00:00:00 ┆ null  ┆ 3.166667 │
│ std       

alt.Chart(...)

Hozzunk létre egy Polars DataFrame-t egy Pandas DataFrame-ből.  
[polars.from_pandas](https://docs.pola.rs/api/python/stable/reference/api/polars.from_pandas.html#polars-from-pandas)  
[polars.DataFrame.to_pandas](https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.to_pandas.html#polars.DataFrame.to_pandas)
 


In [ ]:
#region-> Példa a Polars és Pandas DataFrame-k közötti konverzióra
import pandas as pd
import polars as pl

df_pd = pd.DataFrame({
    'x': [1, 2, 3],
    'y': [4, 5, 6],
    'z': [7, 8, 9]
})


df_pl = pl.from_pandas(df_pd)
df_pd['x'] = [10, 20, 30]

print("A módosított Pandas dataframe:")
print(df_pd)

print("\nA Polars dataframe nem módosul")
print(df_pl)
#endregion


### M01 - első mérkőzés  
![](assets/pandas_vs_polars_h_128.png)  

 
Most hogy lassan ismerkedünk az új szintaxissal, ideje egy kis mérkőzésnek: nyissuk meg a `introduction_to_polars.ipynb` Jupyter notebookot.

## 4. Kifejezések, DataFrame műveletek
Link a dokumentációhoz: [Expressions](https://docs.pola.rs/api/python/stable/reference/expressions/index.html). 


A kifejezések a Polars könyvtár egyik alapeleme, mégis önmagukban a kifejezések nem tesznek semmit.  
A gyakorlatban a kifejezéseket úgy alkalmazzák, hogy átadják őket néhány `DataFrame` vagy `LazyFrame` metódusnak.

Nézünk pár példát a kifejezések használatára:

- Oszlopok kiválasztása a `df.select()` metódussal
- Új oszlopok létrehozása a `df.with_columns()` metódussal
- Sorok szűrése a `df.filter()` metódussal
- Aggregálás a `df.group_by()` metódussal
- Sorok rendezése a `df.sort()` metódussal

A példákhoz használjuk a már jól ismert pingvin datasetet.

### 4.1 Oszlopok kiválasztása, létrehozása

In [91]:
import polars as pl


def get_penguins_df() -> pl.DataFrame:
    """
    
    :return: 
    """
    return pl.read_csv("data/penguins.csv", null_values="NA")


#region-> beolvasás csv fájlból
df = get_penguins_df()
with pl.Config(tbl_cols=-1, tbl_rows=10):
    print(df.schema)
    print(df)
#endregion    


Schema({'rowid': Int64, 'species': String, 'island': String, 'bill_length_mm': Float64, 'bill_depth_mm': Float64, 'flipper_length_mm': Int64, 'body_mass_g': Int64, 'sex': String, 'year': Int64})
shape: (344, 9)
┌───────┬───────────┬───────────┬────────────┬────────────┬────────────┬───────────┬────────┬──────┐
│ rowid ┆ species   ┆ island    ┆ bill_lengt ┆ bill_depth ┆ flipper_le ┆ body_mass ┆ sex    ┆ year │
│ ---   ┆ ---       ┆ ---       ┆ h_mm       ┆ _mm        ┆ ngth_mm    ┆ _g        ┆ ---    ┆ ---  │
│ i64   ┆ str       ┆ str       ┆ ---        ┆ ---        ┆ ---        ┆ ---       ┆ str    ┆ i64  │
│       ┆           ┆           ┆ f64        ┆ f64        ┆ i64        ┆ i64       ┆        ┆      │
╞═══════╪═══════════╪═══════════╪════════════╪════════════╪════════════╪═══════════╪════════╪══════╡
│ 1     ┆ Adelie    ┆ Torgersen ┆ 39.1       ┆ 18.7       ┆ 181        ┆ 3750      ┆ male   ┆ 2007 │
│ 2     ┆ Adelie    ┆ Torgersen ┆ 39.5       ┆ 17.4       ┆ 186        ┆ 3800     

In [ ]:
# Válasszuk ki az összes oszlopot
df.select("*")
df.select(pl.all())

In [74]:
#region-> kifejezések használata a DataFrame.select() metódussal
df.select(
    pl.col("rowid"),
    pl.col("species"),  # fontos, az oszlopok neve case-sensitive
    pl.col("^bill.*$")  # reguláris kifejezés használata az oszlopok kiválasztásra
)
#endregion

rowid,species,bill_length_mm,bill_depth_mm
u16,str,f64,f64
1,"""Adelie""",39.1,18.7
2,"""Adelie""",39.5,17.4
3,"""Adelie""",40.3,18.0
4,"""Adelie""",null,null
5,"""Adelie""",36.7,19.3
…,…,…,…
340,"""Chinstrap""",55.8,19.8
341,"""Chinstrap""",43.5,18.1
342,"""Chinstrap""",49.6,18.2


#### 4.1.1 Szelektorok (`column selectors`) használata

In [63]:
import polars.selectors as cs

df.select(
    cs.by_name("species"),
    cs.starts_with("bill"),
    cs.contains("flipper")
)

# oszlop típusa szerinti kiválasztás
df.select(cs.by_dtype(pl.UInt16))

# oszlopok indexe szerinti kiválasztás
df.select(cs.by_index(range(0,3)))

# szelektorok összekapcsolása
df.select(cs.by_index(range(0,3)) & ~cs.by_dtype(pl.UInt16))


species,island
str,str
"""Adelie""","""Torgersen"""
"""Adelie""","""Torgersen"""
"""Adelie""","""Torgersen"""
"""Adelie""","""Torgersen"""
"""Adelie""","""Torgersen"""
…,…
"""Chinstrap""","""Dream"""
"""Chinstrap""","""Dream"""
"""Chinstrap""","""Dream"""


A szelektor operátorokkal 0 vagy több oszlopot tartalmazó halmazt választunk ki.  



| Szelektor operátor | Inline Operátor | Leírás                          |
|--------------------|-----------------|---------------------------------|
| Unió               | \|              | `x`, vagy `y`, vagy mindkettő   |
| Metszet            | &               | mindkettő, `x` és `y` is        |
| Különbség          | -               | `x`, ami nincs benne `y`-ban    |
| Kizáró VAGY        | ^               | `x`, vagy `y`, de nem mindkettő |
| Negáció            | ~               | Nincs benne `x`-ben             |

### 4.2 Új oszlopok létrehozása

In [146]:
# feltételezzük, hogy a 2009-es méréseket követően a pingvinek szárnyainak 10%-os növekedése várható
# hozzunk létre egy flipper_legth_in_one_year_mm oszlopot, amely a flipper_length_mm oszlop értékének 10%-ával növelt értékeket tartalmazza
df = get_penguins_df()

(df.with_columns(
    # pl.col("flipper_length_mm").mul(1.1).alias("flipper_length_in_one_year_mm"),
    (pl.col("flipper_length_mm") * 1.1).alias("flipper_length_in_one_year_mm"),
    (pl.col("flipper_length_mm") / pl.col("bill_length_mm")).alias("flipper_to_bill_ratio"))
.with_columns(
    bill_length_depth_ratio=pl.col("bill_length_mm") / pl.col("bill_depth_mm"),
    planet=pl.lit("Earth"))
.select(
    pl.col("rowid"),
    pl.col("species"),
    pl.col("bill_length_mm"),
    pl.col("flipper_length_mm"),
    pl.col("flipper_length_in_one_year_mm"),
    pl.col("flipper_to_bill_ratio"),
    pl.col("bill_length_depth_ratio"),
    pl.col("planet")
))


rowid,species,bill_length_mm,flipper_length_mm,flipper_length_in_one_year_mm,flipper_to_bill_ratio,bill_length_depth_ratio,planet
i64,str,f64,i64,f64,f64,f64,str
1,"""Adelie""",39.1,181,199.1,4.629156,2.090909,"""Earth"""
2,"""Adelie""",39.5,186,204.6,4.708861,2.270115,"""Earth"""
3,"""Adelie""",40.3,195,214.5,4.83871,2.238889,"""Earth"""
4,"""Adelie""",null,null,null,null,null,"""Earth"""
5,"""Adelie""",36.7,193,212.3,5.258856,1.901554,"""Earth"""
…,…,…,…,…,…,…,…
340,"""Chinstrap""",55.8,207,227.7,3.709677,2.818182,"""Earth"""
341,"""Chinstrap""",43.5,202,222.2,4.643678,2.403315,"""Earth"""
342,"""Chinstrap""",49.6,193,212.3,3.891129,2.725275,"""Earth"""



### 4.3 Szűrés

Dokumentáció: [df.filter() metódus használata a sorok szűrésére](https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.filter.html)

In [144]:
df.filter(
    pl.col("species") == "Adelie",
    pl.col("island") == "Biscoe",
    pl.col("flipper_length_mm") > 180,
    pl.all().is_not_null()
)

rowid,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
i64,str,str,f64,f64,i64,i64,str,i64
23,"""Adelie""","""Biscoe""",35.9,19.2,189,3800,"""female""",2007
24,"""Adelie""","""Biscoe""",38.2,18.1,185,3950,"""male""",2007
26,"""Adelie""","""Biscoe""",35.3,18.9,187,3800,"""female""",2007
27,"""Adelie""","""Biscoe""",40.6,18.6,183,3550,"""male""",2007
28,"""Adelie""","""Biscoe""",40.5,17.9,187,3200,"""female""",2007
…,…,…,…,…,…,…,…,…
112,"""Adelie""","""Biscoe""",45.6,20.3,191,4600,"""male""",2009
113,"""Adelie""","""Biscoe""",39.7,17.7,193,3200,"""female""",2009
114,"""Adelie""","""Biscoe""",42.2,19.5,197,4275,"""male""",2009


### 4.4 Rendezés

In [130]:

df1 = get_penguins_df()

# Rendezés egy szempont szerint
df1 \
    .filter(pl.col("rowid").is_between(4, 12)) \
    .sort("flipper_length_mm", nulls_last=True)

rowid,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
i64,str,str,f64,f64,i64,i64,str,i64
12,"""Adelie""","""Torgersen""",37.8,17.3,180,3700,null,2007
7,"""Adelie""","""Torgersen""",38.9,17.8,181,3625,"""female""",2007
11,"""Adelie""","""Torgersen""",37.8,17.1,186,3300,null,2007
6,"""Adelie""","""Torgersen""",39.3,20.6,190,3650,"""male""",2007
10,"""Adelie""","""Torgersen""",42.0,20.2,190,4250,null,2007
5,"""Adelie""","""Torgersen""",36.7,19.3,193,3450,"""female""",2007
9,"""Adelie""","""Torgersen""",34.1,18.1,193,3475,null,2007
8,"""Adelie""","""Torgersen""",39.2,19.6,195,4675,"""male""",2007
4,"""Adelie""","""Torgersen""",null,null,null,null,null,2007


In [128]:
# Rendezés több szempont szerint
df1.sort("species", "bill_length_mm", "bill_depth_mm", "flipper_length_mm",
         descending=[False, True, True, True],
         nulls_last=True)


rowid,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
i64,str,str,f64,f64,i64,i64,str,i64
20,"""Adelie""","""Torgersen""",46.0,21.5,194,4200,"""male""",2007
74,"""Adelie""","""Torgersen""",45.8,18.9,197,4150,"""male""",2008
112,"""Adelie""","""Biscoe""",45.6,20.3,191,4600,"""male""",2009
44,"""Adelie""","""Dream""",44.1,19.7,196,4400,"""male""",2007
130,"""Adelie""","""Torgersen""",44.1,18.0,210,4000,"""male""",2009
…,…,…,…,…,…,…,…,…
189,"""Gentoo""","""Biscoe""",42.6,13.7,213,4950,"""female""",2008
169,"""Gentoo""","""Biscoe""",42.0,13.5,210,4150,"""female""",2007
259,"""Gentoo""","""Biscoe""",41.7,14.7,210,4700,"""female""",2009


## 5. Adatok lekérdezése több DataFrame-ből (joins)

| Paraméter     | Leírás                                                                                                                    |
|---------------|---------------------------------------------------------------------------------------------------------------------------|
| `other`       | A DataFrame, amelyhez az aktuális DataFrame-t illesztjük                                                                  |
| `on`          | Az oszlop(ok), amelyeken az illesztést végre kell hajtani, amennyiben az oszlopnév megegyezik a bal és jobb DataFrameben. |
| `left_on` és `right_on` | Az oszlop(ok), amelyeken a csatlakozást végre kell hajtani, ha az oszlopok eltérő nevet viselnek a bal és jobb keretben.  |
| `how`         | Az illesztés típusa                                                                                                       |
| `suffix`      | Az utótag, amely a mindkét DataFrame-be megjelenő oszlopokhoz lesz hozzáadva.                                             |
| `validate`    | Ellenőrzés az illesztéshez                                                                                                |
| `join_nulls`  | Null értékek illesztése. Alapértelmezés szerint a null értékek nem vesszük figyelembe join esetén.                        |


In [180]:
keys_1 = ["K1", "K2", "K3", "K4"]
df1 = pl.DataFrame({
    "key": keys_1,
    "value": np.random.randint(1, 10, len(keys_1))
})
df1

key,value
str,i64
"""K1""",6
"""K2""",2
"""K3""",6
"""K4""",5


In [175]:
keys_2 = ["K2", "K2", "K3", "K4", "K5", "K6"]
df2 = pl.DataFrame({
    "key": keys_2,
    "value": np.random.randint(1, 10, len(keys_2))
})
df2

key,value
str,i64
"""K2""",1
"""K2""",6
"""K3""",6
"""K4""",6
"""K5""",6
"""K6""",5


In [191]:
df1.join(other=df2, on="key", how="inner")

key,value,value_right
str,i64,i64
"""K2""",2,1
"""K2""",2,6
"""K3""",6,6
"""K4""",5,6


In [190]:

df1.join(other=df2, left_on="key", right_on="key", how="inner")

key,value,value_right
str,i64,i64
"""K2""",2,1
"""K2""",2,6
"""K3""",6,6
"""K4""",5,6


In [183]:
df1.join(df2, on="key", how="full", suffix="_other") #korábban outer

key,value,key_other,value_other
str,i64,str,i64
"""K2""",2,"""K2""",1
"""K2""",2,"""K2""",6
"""K3""",6,"""K3""",6
"""K4""",5,"""K4""",6
null,null,"""K5""",6
null,null,"""K6""",5
"""K1""",6,null,null


In [184]:
df1.join(df2, on="key", how="left")

key,value,value_right
str,i64,i64
"""K1""",6,null
"""K2""",2,1
"""K2""",2,6
"""K3""",6,6
"""K4""",5,6


In [187]:
df1.join(df2, on="key", how="semi")

key,value
str,i64
"""K2""",2
"""K3""",6
"""K4""",5


In [189]:
df1.join(df2, on=["key", "value"], how="full")

key,value,key_right,value_right
str,i64,str,i64
null,null,"""K2""",1
null,null,"""K2""",6
"""K3""",6,"""K3""",6
null,null,"""K4""",6
null,null,"""K5""",6
null,null,"""K6""",5
"""K2""",2,null,null
"""K1""",6,null,null
"""K4""",5,null,null


In [151]:

df_movies = pl.read_json("data/movies.json")
print(df_movies.schema)
df_movies.head(10)

Schema({'movie_id': Int64, 'title': String, 'budget': Int64, 'homepage': String, 'overview': String, 'popularity': Float64, 'release_date': String, 'revenue': Int64, 'runtime': Int64, 'movie_status': String, 'tagline': String, 'vote_average': Float64, 'vote_count': Int64})


movie_id,title,budget,homepage,overview,popularity,release_date,revenue,runtime,movie_status,tagline,vote_average,vote_count
i64,str,i64,str,str,f64,str,i64,i64,str,str,f64,i64
5,"""Four Rooms""",4000000,"""""","""It's Ted the Bellhop's first n…",22.87623,"""1995-12-09""",4300000,98,"""Released""","""Twelve outrageous guests. Four…",6.5,530
11,"""Star Wars""",11000000,"""http://www.starwars.com/films/…","""Princess Leia is captured and …",126.393695,"""1977-05-25""",775398007,121,"""Released""","""A long time ago in a galaxy fa…",8.1,6624
12,"""Finding Nemo""",94000000,"""http://movies.disney.com/findi…","""Nemo, an adventurous young clo…",85.688789,"""2003-05-30""",940335536,100,"""Released""","""There are 3.7 trillion fish in…",7.6,6122
13,"""Forrest Gump""",55000000,"""""","""A man with a low IQ has accomp…",138.133331,"""1994-07-06""",677945399,142,"""Released""","""The world will never be the sa…",8.2,7927
14,"""American Beauty""",15000000,"""http://www.dreamworks.com/ab/""","""Lester Burnham, a depressed su…",80.878605,"""1999-09-15""",356296601,122,"""Released""","""Look closer.""",7.9,3313
16,"""Dancer in the Dark""",12800000,"""""","""Selma, a Czech immigrant on th…",22.022228,"""2000-05-17""",40031879,140,"""Released""","""You don't need eyes to see.""",7.6,377
18,"""The Fifth Element""",90000000,"""""","""In 2257, a taxi driver is unin…",109.528572,"""1997-05-07""",263920180,126,"""Released""","""There is no future without it.""",7.3,3885
19,"""Metropolis""",92620000,"""""","""In a futuristic city sharply d…",32.351527,"""1927-01-10""",650422,153,"""Released""","""There can be no understanding …",8.0,657
20,"""My Life Without Me""",0,"""http://www.clubcultura.com/clu…","""A Pedro Almodovar production i…",7.958831,"""2003-03-07""",9726954,106,"""Released""","""""",7.2,77


In [157]:
df_movie_genre = pl.read_json("data/movie_genre.json")
print(df_movie_genre.schema)
df_movie_genre.head(10)

Schema({'movie_id': Int64, 'genre_id': Int64})


movie_id,genre_id
i64,i64
5,35
5,80
11,12
11,28
11,878
12,16
12,10751
13,18
13,35


In [156]:
df_genres = pl.read_json("data/genres.json")
print(df_genres.schema)
df_genres.head(10)

Schema({'genre_id': Int64, 'genre_name': String})


genre_id,genre_name
i64,str
12,"""Adventure"""
14,"""Fantasy"""
16,"""Animation"""
18,"""Drama"""
27,"""Horror"""
28,"""Action"""
35,"""Comedy"""
36,"""History"""
37,"""Western"""


In [159]:
df_combined = (df_movies
.join(df_movie_genre, on='movie_id', how="inner").join(df_genres, on='genre_id')
.select([
    pl.col('movie_id'),
    pl.col('title'),
    pl.col('genre_name')
]))
df_combined


movie_id,title,genre_name
i64,str,str
5,"""Four Rooms""","""Comedy"""
5,"""Four Rooms""","""Crime"""
11,"""Star Wars""","""Adventure"""
11,"""Star Wars""","""Action"""
11,"""Star Wars""","""Science Fiction"""
…,…,…
426469,"""Growing Up Smith""","""Family"""
433715,"""8 Days""","""Drama"""
433715,"""8 Days""","""Thriller"""


In [170]:
df_movies_with_genres = df_combined \
    .group_by('movie_id') \
    .agg([
    pl.col('title').first(),  # Get the movie title (assuming it's the same for each movie_id)
    pl.col('genre_name')  # Collect genres into a list for each movie
]) \
    .filter(pl.col("title") == "Four Rooms")
df_movies_with_genres


movie_id,title,genre_name
i64,str,list[str]
5,"""Four Rooms""","[""Comedy"", ""Crime""]"
